# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 1

user_params = {
    'media_file': f'../flaviolin data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05
A2,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05
A3,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05
A4,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,0.000102,0.000255,0.0001,0.0008,0.0001,0.05
A5,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,0.000102,0.000255,0.0001,0.0008,0.0001,0.05


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [5]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A4    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A5    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A6    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A7    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A8    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B4    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B5    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B6    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B7    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B8    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [6]:
column_order = True if df.index[1][0] == 'B' else False

In [19]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C1_WA1_A3-R1
C1_WA1_A3-R2
C1_WA1_A3-R3
C1_WA4_A6-R1
C1_WA4_A6-R2
C1_WA4_A6-R3
C1_WA7_B1-R1
C1_WA7_B1-R2
C1_WA7_B1-R3
C1_WB2_B4-R1
C1_WB2_B4-R2
C1_WB2_B4-R3
C1_WB5_B7-R1
C1_WB5_B7-R2
C1_WB5_B7-R3
C1_WB8_C2-R1
C1_WB8_C2-R2
C1_WB8_C2-R3
C1_WC3_C5-R1
C1_WC3_C5-R2
C1_WC3_C5-R3
C1_WC6_C8-R1
C1_WC6_C8-R2
C1_WC6_C8-R3
C1_WD1_D3-R1
C1_WD1_D3-R2
C1_WD1_D3-R3
C1_WD4_D6-R1
C1_WD4_D6-R2
C1_WD4_D6-R3
C1_WD7_E1-R1
C1_WD7_E1-R2
C1_WD7_E1-R3
C1_WE2_E4-R1
C1_WE2_E4-R2
C1_WE2_E4-R3
C1_WE5_E7-R1
C1_WE5_E7-R2
C1_WE5_E7-R3
C1_WE8_F2-R1
C1_WE8_F2-R2
C1_WE8_F2-R3
C1_WF3_F5-R1
C1_WF3_F5-R2
C1_WF3_F5-R3
C1_WF6_F8-R1
C1_WF6_F8-R2
C1_WF6_F8-R3


In [20]:
df.head(5)

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA1_A3-R1
A2,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA1_A3-R2
A3,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C1_WA1_A3-R3
A4,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,0.000102,0.000255,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA4_A6-R1
A5,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,0.000102,0.000255,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA4_A6-R2


## Process measurement files

Read measurements file:

In [24]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

# df_600

In [25]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.3113,0.3186,0.3161,0.2992,0.2896,0.2896,0.3023,0.3331,0.1037,0.1006,0.0992,0.0970
B,0.2639,0.4357,0.4083,0.4315,0.3956,0.3792,0.3651,0.9648,0.1009,0.0986,0.0965,0.0926
C,0.7174,0.6943,0.3695,0.3729,0.3707,0.4129,0.4393,0.4546,0.0936,0.0933,0.0898,0.0886
D,0.3874,0.3575,0.3672,0.5828,0.5831,0.6040,0.6728,0.6806,0.0868,0.0859,0.0850,0.0817
E,0.7123,0.3539,0.3587,0.3424,0.4729,0.5074,0.4836,0.2839,0.0821,0.0804,0.0808,0.0765
F,0.2963,0.2847,0.4961,0.4531,0.4429,0.4010,0.4019,0.4015,0.0789,0.0783,0.0752,0.0728
G,0.0723,0.0750,0.0777,0.0797,0.0806,0.0807,0.0803,0.0796,0.0785,0.0771,0.0734,0.0723
H,0.0738,0.0746,0.0887,0.0769,0.0771,0.0762,0.0757,0.0762,0.0764,0.0759,0.0739,0.0719


### Process OD600

Normalize the data to the control well A9 value (water content):

In [26]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.0785,0.0625,0.0723,0.7879,0.7633,0.5798,0.0315,0.0340,0.0000,0.0090,0.0091,0.0087
B,0.0702,0.8223,0.1857,0.4752,0.8696,0.6816,0.6521,0.6576,0.0065,0.0054,0.0056,0.0048
C,0.6150,0.6231,0.8488,0.8557,0.8586,0.9280,0.8583,0.6101,0.0002,0.0014,-0.0002,0.0005
D,1.0580,0.7944,0.7673,0.6066,0.5947,0.6058,0.9231,0.9181,-0.0048,-0.0045,-0.0051,-0.0054
E,0.9459,0.1430,0.4400,0.6560,0.1794,0.8274,0.9285,0.2410,-0.0088,-0.0091,-0.0092,-0.0102
F,0.2515,0.2586,0.5788,0.4340,0.4795,0.7794,0.7380,0.0584,-0.0118,-0.0119,-0.0123,-0.0125
G,-0.0130,-0.0130,-0.0123,-0.0124,-0.0120,-0.0125,-0.0121,-0.0125,-0.0127,-0.0129,-0.0132,-0.0135
H,-0.0122,-0.0122,-0.0119,-0.0118,-0.0120,-0.0123,-0.0123,-0.0124,-0.0123,-0.0120,-0.0125,-0.0127


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [27]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,0.785,0.625,0.723,7.879,7.633,5.798,0.315,0.340
B,0.702,8.223,1.857,4.752,8.696,6.816,6.521,6.576
C,6.150,6.231,8.488,8.557,8.586,9.280,8.583,6.101
D,10.580,7.944,7.673,6.066,5.947,6.058,9.231,9.181
E,9.459,1.430,4.400,6.560,1.794,8.274,9.285,2.410
F,2.515,2.586,5.788,4.340,4.795,7.794,7.380,0.584


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [28]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.2076,0.2149,0.2124,0.1955,0.1859,0.1859,0.1986,0.2294
B,0.1602,0.3320,0.3046,0.3278,0.2919,0.2755,0.2614,0.8611
C,0.6137,0.5906,0.2658,0.2692,0.2670,0.3092,0.3356,0.3509
D,0.2837,0.2538,0.2635,0.4791,0.4794,0.5003,0.5691,0.5769
E,0.6086,0.2502,0.2550,0.2387,0.3692,0.4037,0.3799,0.1802
F,0.1926,0.1810,0.3924,0.3494,0.3392,0.2973,0.2982,0.2978


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [29]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [30]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.019871,20.0,0.205718,2.36508,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA1_A3-R1,0.2076
A2,40.0,4.0,0.019871,20.0,0.205718,2.36508,0.021243,6.375749,4.320423,21.652901,0.000023,0.001306,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA1_A3-R2,0.2149


## Create EDD Experiment Description File

In [31]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [32]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [33]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [34]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [20]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [36]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

In [37]:
df

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],...,Media,Part ID,Culture Volume,Flask Volume,Growth Temperature,Shaking speed,Measurement Type,Time,Value,Units
Well,,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.2076,n/a
A2,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.2149,n/a
A3,40.0,4.0,0.019871,20.0,0.205718,2.365080,0.021243,6.375749,4.320423,21.652901,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.2124,n/a
A4,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.1955,n/a
A5,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.1859,n/a
A6,40.0,4.0,0.003918,20.0,1.883663,0.799089,0.009532,8.860595,3.425574,15.010088,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.1859,n/a
A7,40.0,4.0,0.000928,20.0,0.114704,0.749104,0.024392,8.910734,3.278069,39.314072,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.1986,n/a
A8,40.0,4.0,0.000928,20.0,0.114704,0.749104,0.024392,8.910734,3.278069,39.314072,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.2294,n/a
B1,40.0,4.0,0.000928,20.0,0.114704,0.749104,0.024392,8.910734,3.278069,39.314072,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,0.1602,n/a
